For each point in the range of emerald ash borrer select all days when the temperature came close to or fell below the supercooling point of EAB.

More formally: select all days when the temperature fell below -30 degrees celsius.

Inputs:
* the range of emerald ash borrer (csv)
    * `EAB range.txt`
* dayly average temperatures on a 0.5 by 0.5 degree grid for a ragne of dates 2003-01-01 through 2019-10-01 (NetCDF)
    * `41fe1db9-74cf-4842-a8ea-f8e55281c9ad.nc`
    * `a45203bd-8884-4178-a068-5d3e5c05e72e.nc`
    * `d6867180-6444-4fd0-bda7-480197f76239.nc`
    
Outputs:
*  where, when and how low did the temperature fall (csv)
    * `result.txt`
    
Notes:

Dayly mean temperature were calculated as an average of temperatures at 00:00, 02:00, 04:00, 06:00, 08:00, 10:00, 12:00, 14:00, 16:00, 18:00, 20:00 and 22:00 UTC.
In the interest of efficient storage and transfer the input does not contain temperature data for the entire Earth.

In [1]:
LOW_TEMP = -30 # celsius

In [2]:
import xarray as xr
import pandas as pd
from IPython.display import FileLink

In [52]:
eab_range = pd.read_csv('EAB range.txt')[['Country', 'Region', 'Locality', 'Latitude', 'Longitude']]

In [53]:
temperature = (
    '41fe1db9-74cf-4842-a8ea-f8e55281c9ad.nc',
    'a45203bd-8884-4178-a068-5d3e5c05e72e.nc',
    'd6867180-6444-4fd0-bda7-480197f76239.nc',
)

In [54]:
# The dataset is read lazily to avoid running out of memory
kelvin = xr.open_mfdataset(temperature, combine='by_coords', chunks={'lon':10})
celsius = kelvin - 273.15
near_surface_temperatures = celsius['tas']

In [55]:
lat = xr.DataArray(eab_range.Latitude, coords={'location': eab_range.index}, dims=['location'])
lon = xr.DataArray(eab_range.Longitude, coords={'location': eab_range.index}, dims=['location'])
temperatures_at_locations = near_surface_temperatures.sel(lat=lat, lon=lon, method='nearest')
temperatures_at_locations = temperatures_at_locations.stack(index=('time', 'location'))

In [56]:
temperatures_at_locations = temperatures_at_locations.load() # Converting to eager before filtering improves performance
temperatures_at_locations = temperatures_at_locations[temperatures_at_locations < LOW_TEMP]
ds = xr.Dataset({'temperature': temperatures_at_locations, 'index': temperatures_at_locations.index})
del ds['realization']

In [57]:
df = ds.to_dataframe()
df = df.reset_index('time')
df = df.rename(columns={'lat': 'nearest_lat', 'lon': 'nearest_lon'})
result = eab_range.join(df, how='right')[['Country', 'Region', 'Locality', 'time', 'temperature', 'Latitude', 'Longitude',  'nearest_lat', 'nearest_lon']]
result = result.sort_values(['Country', 'Region', 'Locality', 'time'])
result

,Country,Region,Locality,time,temperature,Latitude,Longitude,nearest_lat,nearest_lon
402,Canada,Manitoba,Winnipeg,2004-01-04,-30.188614,49.913721,-97.197488,50.0,-97.0
402,Canada,Manitoba,Winnipeg,2004-01-05,-32.513092,49.913721,-97.197488,50.0,-97.0
402,Canada,Manitoba,Winnipeg,2004-01-29,-34.694000,49.913721,-97.197488,50.0,-97.0
402,Canada,Manitoba,Winnipeg,2004-01-30,-37.751877,49.913721,-97.197488,50.0,-97.0
402,Canada,Manitoba,Winnipeg,2004-01-31,-32.703568,49.913721,-97.197488,50.0,-97.0
...,...,...,...,...,...,...,...,...,...
399,USA,Minnesota,NaN,2019-01-31,-30.835007,44.958255,-93.154602,45.0,-93.0
400,USA,Minnesota,NaN,2019-01-31,-30.835007,45.031186,-93.109344,45.0,-93.0
401,USA,Minnesota,NaN,2019-01-31,-30.835007,45.179624,-93.085355,45.0,-93.0
415,USA,South Dakota,NaN,2009-01-15,-30.364960,43.614580,-96.787122,43.5,-97.0


In [58]:
result.to_csv('result.txt', index=False)
FileLink('result.txt')

/home/user/result.txt

Obtaining the temperature data
==============================
The files used in this analysis can be downloaded from here:
* https://github.com/hexagonrecursion/supercooling/releases/download/1.0/41fe1db9-74cf-4842-a8ea-f8e55281c9ad.nc
* https://github.com/hexagonrecursion/supercooling/releases/download/1.0/a45203bd-8884-4178-a068-5d3e5c05e72e.nc
* https://github.com/hexagonrecursion/supercooling/releases/download/1.0/d6867180-6444-4fd0-bda7-480197f76239.nc

They were generated by submitting the following query to https://cds.climate.copernicus.eu/

In short: this query calculates dayly mean temperature as an average of temperatures at 00:00, 02:00, 04:00, 06:00, 08:00, 10:00, 12:00, 14:00, 16:00, 18:00, 20:00 and 22:00 UTC.

```python
START_YEAR = 2003
END_YEAR = 2019
AREA = [
    [10, -140, 65, -50],
    [20, 100, 60, 150],
    [30, -20, 75, 70],
]

import cdstoolbox as ct

@ct.application(title=f'Dayly means')
@ct.output.download()
@ct.output.download()
@ct.output.download()
def daily_means():
    years = [str(y) for y in range(START_YEAR, END_YEAR + 1)]
    print(years)
    
    result = []
    for area in AREA:
        data = ct.catalogue.retrieve(
        'reanalysis-era5-land',
        {
            'area': area,
            'grid': ['0.5', '0.5'],
            'variable': '2m_temperature',
            'year': years,
            'month': [
                '01', '02', '03', '04', '05', '06',
                '07', '08', '09', '10', '11', '12'
            ],
            'day': [
                '01', '02', '03', '04', '05', '06',
                '07', '08', '09', '10', '11', '12',
                '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24',
                '25', '26', '27', '28', '29', '30',
                '31'
            ],
            'time': [
                '00:00', '02:00', '04:00',
                '06:00', '08:00', '10:00',
                '12:00', '14:00', '16:00',
                '18:00', '20:00', '22:00',
            ],
        }
        )
    
        daily_means = ct.climate.daily_mean(data)
        del data
    
        result.append(daily_means)

    return result[0], result[1], result[2]
```

If you intend to replicate the retrieval of the temperature data note that:
1. The query will take a long time (hours) because the data needs to be retrieved from tape
2. The file names will likely differ
3. At the time of writing the query does not return the whole date range specified by the query: only 2003-01-01 through 2019-10-01. In the future the full range may be returned.

Copyright
=========

The followig files were generated using Copernicus Climate Change Service Information 2020; see https://apps.ecmwf.int/datasets/licences/copernicus/:
* `41fe1db9-74cf-4842-a8ea-f8e55281c9ad.nc`
* `a45203bd-8884-4178-a068-5d3e5c05e72e.nc`
* `d6867180-6444-4fd0-bda7-480197f76239.nc`

The followig files were autohored by Copyright (c) 2020 Andrey Bienkowski <hexagonrecursion@gmail.com>:
* `supercooling.ipynb`
* `README.txt`
* `environment.yml`
* `postBuild`

MIT License

Copyright (c) 2020 Andrey Bienkowski <hexagonrecursion@gmail.com>

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

For the file `EAB range.txt` see the `Source of information` column of `EAB range.txt`